## 🧹 Cleaning Imperfect Data Structure (Headers/Columns)

This notebook shows how to fix **imperfect data structure** in maintenance logs (e.g., `log_data.xlsx`).

### 🎯 What you'll learn
- Normalize messy headers (remove quotes/newlines, trim spaces)
- Standardize header style (lowercase, underscores)
- Tidy inconsistent text fields (case, extra spaces)

#### ⚙️ Dataset
Place your file **`log_data.xlsx`** in the raw_data folder.

In [6]:
# 📦 Imports
import pandas as pd
import re
from pathlib import Path

path = Path('raw_data/log_data.xlsx')
assert path.exists(), 'Place log_data.xlsx in the data folder before running.'

# Read first sheet by default
df = pd.read_excel(path)
print('Loaded shape:', df.shape)
df.head(3)

Loaded shape: (21, 26)


,line,MCCE\nquipment,MCCDescription,ProblemsItems,Action,jobcompleted,Shift,Month,IssueDate,EndDate,...,M C C,PersonFinishJob,SpareStatusandorigin-from,SAPNo,SAPCode,SpareParts,quantity,LE/Uintes,PMCM,Reason
0,Line 4,0405PP03,طلمبة الزيت الرئيسية,فك طلمبة الزيت وتركيب ميكانيكال سيل,تم التركيب,Yes,2,January,2020-01-15,2020-01-15,...,9350.0,Ahmed Abdullah,Warehouse,6.902682e+09,860010601.0,"Seal Pump 2.5"" -‏ BME119A-A00 (20503157)",1.0,70952.33,CM,Technical
1,Line 4,0405pp03,طلمبة الزيت الرئيسية,كاوتشه كوبلنج,تم التركيب,Yes,2,January,2020-01-11,2020-01-12,...,9347.0,Saleh Ibrahim,Warehouse,6.902590e+09,860076793.0,"Shaft Coupling 2.5""-75mm F120 (20528660)",1.0,14030.43,CM,Technical
2,Line 4,0405pp03,طلمبة الزيت الرئيسية,ميكانيكال سيل ص أ,تم التركيب,Yes,2,January,2020-01-11,2020-01-12,...,9347.0,Saleh Ibrahim,NaN,NaN,NaN,ميكانيكال سيل ض أ,1.0,NaN,CM,Technical


#### 🔎 Inspect Raw Headers
Messy exports often contain **newlines** (`\n`), **quotes** (`"`), **extra spaces**, and **inconsistent case** in column names.

In [7]:
raw_cols = list(df.columns)
raw_cols

['line',
 'MCCE\nquipment',
 'MCCDescription',
 'ProblemsItems',
 'Action',
 'jobcompleted',
 'Shift',
 'Month',
 'IssueDate',
 'EndDate',
 'Starttime',
 'Finishtime',
 'NetTime',
 'D.T Time',
 'R.T Time',
 'W O',
 'M C C',
 'PersonFinishJob',
 'SpareStatusandorigin-from',
 'SAPNo',
 'SAPCode',
 'SpareParts',
 'quantity',
 'LE/Uintes',
 'PMCM',
 'Reason']

#### 🛠️ Header Normalization
We will:
1. Remove quotes
2. Replace newlines and multiple spaces with single underscore
3. Trim whitespace
4. Convert to lowercase
5. Replace remaining spaces with underscores

**Tip:** keeping headers `snake_case` makes downstream Python analytics much easier.

In [8]:
def clean_header(col: str) -> str:
    if not isinstance(col, str):
        col = str(col)
    col = col.replace('"', '')  # remove quotes
    col = col.replace("'", '')  # remove single quotes
    col = col.replace('\r', ' ').replace('\n', ' ')  # remove newlines
    col = re.sub(r'\s+', ' ', col)  # collapse whitespace
    col = col.strip().lower()  # trim + lowercase
    col = col.replace(' ', '_')  # spaces -> underscores
    # remove non-alnum/underscore except Arabic letters
    col = re.sub(r'[^0-9a-zA-Z_\u0600-\u06FF]', '', col)
    # collapse multiple underscores
    col = re.sub(r'_+', '_', col)
    return col

cleaned_cols = [clean_header(c) for c in df.columns]
cleaned_cols

['line',
 'mcce_quipment',
 'mccdescription',
 'problemsitems',
 'action',
 'jobcompleted',
 'shift',
 'month',
 'issuedate',
 'enddate',
 'starttime',
 'finishtime',
 'nettime',
 'dt_time',
 'rt_time',
 'w_o',
 'm_c_c',
 'personfinishjob',
 'sparestatusandoriginfrom',
 'sapno',
 'sapcode',
 'spareparts',
 'quantity',
 'leuintes',
 'pmcm',
 'reason']

#### ✅ Apply Cleaned Headers
We'll assign the cleaned names back to the DataFrame and preview the result.

In [9]:
df.columns = cleaned_cols
df.head(3)

,line,mcce_quipment,mccdescription,problemsitems,action,jobcompleted,shift,month,issuedate,enddate,...,m_c_c,personfinishjob,sparestatusandoriginfrom,sapno,sapcode,spareparts,quantity,leuintes,pmcm,reason
0,Line 4,0405PP03,طلمبة الزيت الرئيسية,فك طلمبة الزيت وتركيب ميكانيكال سيل,تم التركيب,Yes,2,January,2020-01-15,2020-01-15,...,9350.0,Ahmed Abdullah,Warehouse,6.902682e+09,860010601.0,"Seal Pump 2.5"" -‏ BME119A-A00 (20503157)",1.0,70952.33,CM,Technical
1,Line 4,0405pp03,طلمبة الزيت الرئيسية,كاوتشه كوبلنج,تم التركيب,Yes,2,January,2020-01-11,2020-01-12,...,9347.0,Saleh Ibrahim,Warehouse,6.902590e+09,860076793.0,"Shaft Coupling 2.5""-75mm F120 (20528660)",1.0,14030.43,CM,Technical
2,Line 4,0405pp03,طلمبة الزيت الرئيسية,ميكانيكال سيل ص أ,تم التركيب,Yes,2,January,2020-01-11,2020-01-12,...,9347.0,Saleh Ibrahim,NaN,NaN,NaN,ميكانيكال سيل ض أ,1.0,NaN,CM,Technical


#### 💾 Save Cleaned Output

In [10]:
out_csv = 'preprocessed_data/log_data_clean_headers.csv'
df.to_csv(out_csv, index=False)